In [1]:
import pandas as pd
import numpy as np
from functools import reduce

from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder

import lightgbm as lgb

from tqdm import tqdm, trange

import numpy as np
np.random.seed(0)

import matplotlib.pyplot as plt

from sklearn.calibration import calibration_curve
from sklearn.calibration import IsotonicRegression

import xgboost as xgb
from catboost import CatBoostClassifier, Pool

from sklearn.model_selection import StratifiedKFold

import mlflow
from utils import log_mlflow, create_submission, apply_isotonic_regression


In [2]:
experiment_name = "criteo_privacy_preserving_competition"
mlflow.set_experiment(experiment_name)

In [3]:
X_train = pd.read_csv('output/X_train_full.csv')
X_test = pd.read_csv('output/X_test_full.csv')

DATA_DIR = 'data/'
Y_TRAIN_PATH = DATA_DIR + 'y_train.csv.gz'
y_train = pd.read_csv(Y_TRAIN_PATH, compression='gzip')

In [9]:
class DatasetHandler:
    
    def __init__(self, X_train, X_valid, y_train, y_valid, X_test):
        self.X_train = X_train
        self.X_valid = X_valid
        self.y_train = y_train
        self.y_valid = y_valid
        self.X_test = X_test
                
    @staticmethod
    def smoothed_stat(df, smoothed_factor, prefix):
        df[f'{prefix}_ctr'] = (smoothed_factor['ctr'] * 100 + df[f'{prefix}_clicks']) / (100 + df[f'{prefix}_count'])
        df[f'{prefix}_cr'] = (smoothed_factor['cr'] * 100 + df[f'{prefix}_sales']) / (100 + df[f'{prefix}_count'])
        df[f'{prefix}_ccr'] = (smoothed_factor['ccr'] * 100 + df[f'{prefix}_sales']) /(100 + df[f'{prefix}_clicks'])
        return df
    
    def get_smoothed_factor(self):
        smoothed_factor = {
            'count': 0, 
            'clicks': 0, 
            'sales': 0,
            'freq': 0,
            'ctr': np.mean(self.y_train['click']),
            'cr': np.mean(self.y_train['sale']),
            'ccr': np.mean(self.y_train[self.y_train['click'] == 1]['sale']) 
        } 
        return smoothed_factor
                
    def preprocess_dataframes(self):

        hash_cols = {f'hash_{i}' for i in range(19)}
        train_cols = list(set(self.X_train.columns) - hash_cols)
        smoothed_factor = self.get_smoothed_factor()

        for dataset in (self.X_train, self.X_valid, self.X_test):
            for i in trange(19):
                dataset = self.smoothed_stat(dataset, smoothed_factor, f'feature{i}')
                for j in trange(19):
                    if i < j:
                        dataset = self.smoothed_stat(dataset, smoothed_factor, f'feature{i}_feature{j}')

            for col in train_cols:
                stat = col.split('_')[-1]
                if stat in smoothed_factor:
                    dataset[col] = dataset[col].replace([np.inf, -np.inf], np.nan).fillna(smoothed_factor[stat])

        dataset_dict = {
            'X_train': self.X_train[train_cols], 
            'X_valid': self.X_valid[train_cols], 
            'y_train': self.y_train['click'], 
            'y_valid': self.y_valid['click'], 
            'X_test':  self.X_test[train_cols],
        }
        return dataset_dict
        

In [17]:
class ModelStacking:
    
    def __init__(self, models, models_params):
        self.models = models
        self.models_params = models_params
        
        self.scores = {model: [] for model in self.models}
        self.val_preds = {model: {} for model in self.models}
        self.test_preds = {model: pd.DataFrame() for model in self.models}
        self.calibrated_test_preds = {model: pd.DataFrame() for model in self.models}
        
    def set_dataset(self, X_train, y_train, X_valid, y_valid, X_test):
        self.X_train = X_train
        self.y_train = y_train
        self.X_valid = X_valid
        self.y_valid = y_valid
        self.X_test = X_test

    def fit_model(self, model_name):
        if model_name == 'lightgbm':
            tr = lgb.Dataset(np.array(self.X_train), np.array(self.y_train))
            te = lgb.Dataset(np.array(self.X_valid), np.array(self.y_valid), reference=tr)
            model = lgb.train(
                self.models_params[model_name], 
                tr, 
                num_boost_round=10000, 
                valid_sets=te, 
                early_stopping_rounds=int(5 / self.models_params[model_name]['learning_rate']), 
                verbose_eval=100,
            )
            self.scores[model_name].append(model.best_score['valid_0']['binary_logloss'])
            self.val_preds[model_name][str(i)] = model.predict(self.X_valid)
            self.test_preds[model_name][str(i)] = model.predict(self.X_test)
        
        elif model_name == 'xgboost':
            model = xgb.XGBClassifier(**models_params[model_name])
            model.fit(
                self.X_train, 
                self.y_train, 
                early_stopping_rounds=int(5 / models_params[model_name]['learning_rate']), 
                eval_metric="logloss", 
                eval_set=[(self.X_valid, self.y_valid)], 
                verbose=100,
            )
            self.scores[model_name].append(model.best_score)
            self.val_preds[model_name][str(i)] = model.predict_proba(self.X_valid)[:, 1]
            self.test_preds[model_name][str(i)] = model.predict_proba(self.X_test)[:, 1]
        
        elif model_name == 'catboost':
            model = CatBoostClassifier(**models_params[model_name])
            model.fit(
                self.X_train, 
                self.y_train, 
                eval_set=Pool(self.X_valid, self.y_valid), 
                verbose=100,
            )
            self.scores[model_name].append(model.best_score_['validation']['Logloss'])
            self.val_preds[model_name][str(i)] = model.predict_proba(self.X_valid)[:, 1]
            self.test_preds[model_name][str(i)] = model.predict_proba(self.X_test)[:, 1]

    def apply_isotonic_regression(self, model_name):
        self.calibrated_test_preds[model_name][str(i)] = apply_isotonic_regression(
            self.val_preds[model_name][str(i)], 
            self.y_valid, 
            self.test_preds[model_name][str(i)],
        )
                        
    def predict(self):
        predictions = reduce(lambda x, y: x+y, [model_stacking.calibrated_test_preds[model_name].mean(axis=1) for model_name in model_stacking.models]) / len(model_stacking.models)
        return predictions


In [11]:
models = {'lightgbm', 'catboost', 'xgboost'}

models_params = {
    'lightgbm': {
        'bagging_fraction': 0.8,
        'bagging_freq': 1,
        'boost': 'gbdt',
        'feature_fraction': 0.8,
        'learning_rate': 0.01,
        'metric':'binary',
        'num_leaves': 31,
        'num_threads': 8,
        'objective': 'binary',
        'seed': 42,
    }, 
    'catboost': {        
        'iterations': 10000,
        'learning_rate': 0.01,
        'loss_function': 'Logloss',
        'early_stopping_rounds': 500,
        'random_seed': 42,
    }, 
    'xgboost': {
        'subsample': 0.8,
        'colsample_bytree': 0.8,
        'learning_rate': 0.01,
        'max_leaves': 31,
        'objective': 'binary:logistic',
        'n_estimators': 10000,
        'use_label_encoder': False,
        'seed': 42,
    },
}


In [19]:
model_stacking = ModelStacking(models, models_params)

kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
for i , (train_index, test_index) in enumerate(kf.split(X_train, y_train['click'])):
    
    data_handler = DatasetHandler(
        X_train.loc[train_index], 
        X_train.loc[test_index], 
        y_train.loc[train_index], 
        y_train.loc[test_index], 
        X_test,
    )
    dataset_dict = data_handler.preprocess_dataframes()
    model_stacking.set_dataset(**dataset_dict)
    
    for model_name in models:
        model_stacking.fit_model(model_name)
        model_stacking.apply_isotonic_regression(model_name)

metrics = {model: np.mean(val_score) for model, val_score in model_stacking.scores.items()}

100%|██████████| 19/19 [00:00<00:00, 253.56it/s]

  0%|          | 0/19 [00:00<?, ?it/s]/usr/local/conda/envs/jupyter/lib/python3.7/site-packages/ipykernel_launcher.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  
/usr/local/conda/envs/jupyter/lib/python3.7/site-packages/ipykernel_launcher.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  if sys.path[0] == '':
/usr/local/conda/envs/jupyter/lib/python3.7/site-packages/ipykernel_launcher.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Co

[LightGBM] [Info] Number of positive: 9170, number of negative: 83020
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.258173 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 123135
[LightGBM] [Info] Number of data points in the train set: 92190, number of used features: 760
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.099468 -> initscore=-2.203144
[LightGBM] [Info] Start training from score -2.203144
Training until validation scores don't improve for 500 rounds
[100]	valid_0's binary_logloss: 0.27063
[200]	valid_0's binary_logloss: 0.254259
[300]	valid_0's binary_logloss: 0.246207
[400]	valid_0's binary_logloss: 0.242198
[500]	valid_0's binary_logloss: 0.240119
[600]	valid_0's binary_logloss: 0.239087
[700]	valid_0's binary_logloss: 0.238469
[800]	valid_0's binary_logloss: 0.238037
[900]	valid_0's binary_logloss: 0.237623
[1000]	valid_0's binary_logloss: 0.237357
[1100]	valid_0's binary_loglo

  0%|          | 0/19 [00:00<?, ?it/s]/usr/local/conda/envs/jupyter/lib/python3.7/site-packages/ipykernel_launcher.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  
/usr/local/conda/envs/jupyter/lib/python3.7/site-packages/ipykernel_launcher.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  if sys.path[0] == '':
/usr/local/conda/envs/jupyter/lib/python3.7/site-packages/ipykernel_launcher.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragm

[LightGBM] [Info] Number of positive: 9170, number of negative: 83020
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.236884 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 123132
[LightGBM] [Info] Number of data points in the train set: 92190, number of used features: 760
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.099468 -> initscore=-2.203144
[LightGBM] [Info] Start training from score -2.203144
Training until validation scores don't improve for 500 rounds
[100]	valid_0's binary_logloss: 0.270579
[200]	valid_0's binary_logloss: 0.253587
[300]	valid_0's binary_logloss: 0.244989
[400]	valid_0's binary_logloss: 0.240462
[500]	valid_0's binary_logloss: 0.23796
[600]	valid_0's binary_logloss: 0.236258
[700]	valid_0's binary_logloss: 0.235207
[800]	valid_0's binary_logloss: 0.234597
[900]	valid_0's binary_logloss: 0.234162
[1000]	valid_0's binary_logloss: 0.233784
[1100]	valid_0's binary_loglo

5900:	learn: 0.1842265	test: 0.2314586	best: 0.2314501 (5876)	total: 5m 43s	remaining: 3m 58s
6000:	learn: 0.1834364	test: 0.2314318	best: 0.2314296 (5998)	total: 5m 49s	remaining: 3m 52s
6100:	learn: 0.1826765	test: 0.2314094	best: 0.2314059 (6097)	total: 5m 55s	remaining: 3m 47s
6200:	learn: 0.1819289	test: 0.2313894	best: 0.2313810 (6195)	total: 6m 1s	remaining: 3m 41s
6300:	learn: 0.1811727	test: 0.2314355	best: 0.2313810 (6195)	total: 6m 8s	remaining: 3m 36s
6400:	learn: 0.1803973	test: 0.2314704	best: 0.2313810 (6195)	total: 6m 14s	remaining: 3m 30s
6500:	learn: 0.1796284	test: 0.2314874	best: 0.2313810 (6195)	total: 6m 20s	remaining: 3m 24s
6600:	learn: 0.1789090	test: 0.2315075	best: 0.2313810 (6195)	total: 6m 26s	remaining: 3m 18s
Stopped by overfitting detector  (500 iterations wait)

bestTest = 0.2313810198
bestIteration = 6195

Shrink model to first 6196 iterations.


...

[LightGBM] [Info] Number of positive: 9170, number of negative: 83021
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.293241 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 123122
[LightGBM] [Info] Number of data points in the train set: 92191, number of used features: 760
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.099467 -> initscore=-2.203156
[LightGBM] [Info] Start training from score -2.203156
Training until validation scores don't improve for 500 rounds
[100]	valid_0's binary_logloss: 0.269618
[200]	valid_0's binary_logloss: 0.252524
[300]	valid_0's binary_logloss: 0.244077
[400]	valid_0's binary_logloss: 0.239614
[500]	valid_0's binary_logloss: 0.237223
[600]	valid_0's binary_logloss: 0.235655
[700]	valid_0's binary_logloss: 0.234737
[800]	valid_0's binary_logloss: 0.234135
[900]	valid_0's binary_logloss: 0.233809
[1000]	valid_0's binary_logloss: 0.233586
[1100]	valid_0's binary_logl

100%|██████████| 19/19 [00:00<00:00, 373.48it/s]

100%|██████████| 19/19 [00:05<00:00,  3.78it/s]


[LightGBM] [Info] Number of positive: 9170, number of negative: 83021
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.252954 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 123172
[LightGBM] [Info] Number of data points in the train set: 92191, number of used features: 760
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.099467 -> initscore=-2.203156
[LightGBM] [Info] Start training from score -2.203156
Training until validation scores don't improve for 500 rounds
[100]	valid_0's binary_logloss: 0.271283
[200]	valid_0's binary_logloss: 0.25516
[300]	valid_0's binary_logloss: 0.247683
[400]	valid_0's binary_logloss: 0.243841
[500]	valid_0's binary_logloss: 0.241364
[600]	valid_0's binary_logloss: 0.240012
[700]	valid_0's binary_logloss: 0.239214
[800]	valid_0's binary_logloss: 0.238623
[900]	valid_0's binary_logloss: 0.23813
[1000]	valid_0's binary_logloss: 0.237836
[1100]	valid_0's binary_loglos

100%|██████████| 19/19 [00:00<00:00, 250.20it/s]

  0%|          | 0/19 [00:00<?, ?it/s]/usr/local/conda/envs/jupyter/lib/python3.7/site-packages/ipykernel_launcher.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  
/usr/local/conda/envs/jupyter/lib/python3.7/site-packages/ipykernel_launcher.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  if sys.path[0] == '':
/usr/local/conda/envs/jupyter/lib/python3.7/site-packages/ipykernel_launcher.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Co

[LightGBM] [Info] Number of positive: 9170, number of negative: 83021
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.383996 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 123098
[LightGBM] [Info] Number of data points in the train set: 92191, number of used features: 760
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.099467 -> initscore=-2.203156
[LightGBM] [Info] Start training from score -2.203156
Training until validation scores don't improve for 500 rounds
[100]	valid_0's binary_logloss: 0.269985
[200]	valid_0's binary_logloss: 0.25272
[300]	valid_0's binary_logloss: 0.244562
[400]	valid_0's binary_logloss: 0.240401
[500]	valid_0's binary_logloss: 0.238084
[600]	valid_0's binary_logloss: 0.23678
[700]	valid_0's binary_logloss: 0.23596
[800]	valid_0's binary_logloss: 0.235516
[900]	valid_0's binary_logloss: 0.235108
[1000]	valid_0's binary_logloss: 0.234853
[1100]	valid_0's binary_logloss

In [20]:
y_hat_click = model_stacking.predict()

# create zip file to upload
assert X_test.shape[0] == y_hat_click.shape[0], \
    "invalid prediction shape: %s expected %s" % (X_test.shape[0], y_hat_click.shape[0])
submission_name = create_submission(y_hat_click, description='lightgbm submission')


wrote submissions/submission-2021-10-20_18-27-12.599036.zip


In [21]:
tags = {
    'model': 'lightgbm',
    'experiment_name': 'stacking',
    'submission_name': submission_name
}

features_path = mlflow.get_artifact_uri() + 'features.csv'
X_train.columns.to_series().to_csv(features_path)
            
log_mlflow(metrics=metrics, tags=tags, parameters=models_params, artifacts=mlflow.get_artifact_uri())

None {'model': 'lightgbm', 'experiment_name': 'stacking', 'submission_name': 'submissions/submission-2021-10-20_18-27-12.599036.zip'} {'lightgbm': {'bagging_fraction': 0.8, 'bagging_freq': 1, 'boost': 'gbdt', 'feature_fraction': 0.8, 'learning_rate': 0.01, 'metric': 'binary', 'num_leaves': 31, 'num_threads': 8, 'objective': 'binary', 'seed': 42}, 'catboost': {'iterations': 10000, 'learning_rate': 0.01, 'loss_function': 'Logloss', 'early_stopping_rounds': 500, 'random_seed': 42}, 'xgboost': {'subsample': 0.8, 'colsample_bytree': 0.8, 'learning_rate': 0.01, 'max_leaves': 31, 'objective': 'binary:logistic', 'n_estimators': 10000, 'use_label_encoder': False, 'seed': 42}} {'lightgbm': 0.235953855612544, 'xgboost': 0.2358611, 'catboost': 0.23579339555420065}
